In this test, I aim to verify the correctness of the `ClassicalShadow_1_CLIFFORD` approximation. To do this, I will override the `get_random_rotations` method. Specifically, it will be modified to always return the Identity operator. When this is applied to a simple $|0\rangle$ state, it becomes straightforward to deterministically calculate the density matrix that the shadow should approximate.

In [ ]:
# Imports

sys.path.insert(0, "../../..")
import sys

from qiskit import QuantumCircuit, qasm2
from qiskit.quantum_info import Clifford, StabilizerState
from qiskit.visualization import array_to_latex
from qiskit_aer import AerSimulator


import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Clifford
from qiskit_aer import AerSimulator

from classical_shadow_1_clifford import ClassicalShadow_1_CLIFFORD
from shadow_protocol import ShadowProtocol

from classical_shadow_1_clifford import ClassicalShadow_1_CLIFFORD
from shadow_protocol import ShadowProtocol

In [ ]:
class Always_id_classical_shadow_1_clifford(ClassicalShadow_1_CLIFFORD):
    def get_random_rotations(self, num_qubits) -> list[Clifford]:
        c_z = Clifford(QuantumCircuit(1))  # Identity
        return [(c_z) for _ in range(num_qubits)]
    

# Setup Experiment
class Id_Protocol(ShadowProtocol):

    def get_num_qubits(self) -> int:
        return 2
    
    def get_state_circuit(self) -> QuantumCircuit:
        circuit = QuantumCircuit(2)
        return circuit

    def run_cuircuit_and_get_measurment(self, circuit) -> list[int]:
        sim = AerSimulator()

        # Run with 997 shots
        job = sim.run(circuit, shots=997)
        result = job.result()

        counts = result.get_counts()
        max_hits= max(counts, key=counts.get)
        bit_list =  [int(bit) for bit in list(max_hits)]
        return bit_list[::-1] 
    
id_protocol = Id_Protocol()
# Create Classical Shadow Instance
always_id = Always_id_classical_shadow_1_clifford(id_protocol)


In [ ]:
found_clifford = {}

for cliff in always_id.get_random_rotations(1000):
    circuit=cliff.to_circuit()
    program_as_string = qasm2.dumps(circuit)
    found_clifford[program_as_string]=cliff

cliffs = list(found_clifford.values())
assert len(cliffs) == 1
display(cliffs[0].to_circuit().draw("mpl"))

This means we will always apply the Identity rotation ($I$), so the state $\rho$ remains unchanged before measurement in the computational basis. For our experiment, the initial state is:

In [ ]:
circuit = always_id.shadow_protocol.get_state_circuit()
display(circuit.draw("mpl"))

dm = always_id.get_original_density_matrix()
display(array_to_latex(dm, prefix="Rho = "))

Putting this together, we expect a circuit that simply measures the qubits without applying any unitary gates beforehand.

In [ ]:

state_circuit = always_id.shadow_protocol.get_state_circuit()
cliffords = always_id.get_random_rotations(2) # will always be the same as showed before

combined_circuit: QuantumCircuit = always_id.make_rotated_state_circuit(
            cliffords, state_circuit
        )
display(combined_circuit.draw("mpl"))

For this circuit, we expect to consistently observe the measurement outcome `00`.

In [ ]:
output = set()
for i in range(0,100):
    measurment = always_id.shadow_protocol.run_cuircuit_and_get_measurment(combined_circuit)
    output.add(str(measurment))

assert len(output) == 1
print(list(output)[0])

If we always measure `00`, the reconstructed snapshot (back-rotated state) can be calculated deterministically using the inverse channel map.

In [ ]:
stabilizers: list[StabilizerState] = (
            always_id.compute_clifford_applied_to_measurements(
                cliffords, [0,0]
            )
        )
for i , stab in enumerate(stabilizers):
    dm = always_id.stabilizer_to_density_matrix(stab)
    display(array_to_latex(dm, prefix=f"DM {i}= "))


Since we always measure the same output, we can analytically determine the resulting matrices. We define them as follows:

$$
DM_0 = DM_1 =
\begin{pmatrix}
1 & 0\\
0 & 0
\end{pmatrix},
\quad
I =
\begin{pmatrix}
1 & 0\\
0 & 1
\end{pmatrix}
$$

We calculate the single-qubit snapshot matrix $A$:

$$
A = 3 DM_0 - I
= 3 \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix} - \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}
= \begin{pmatrix} 2 & 0 \\ 0 & -1 \end{pmatrix}
$$

Finally, we calculate the tensor product $A \otimes A$ for the two-qubit system:

$$
A \otimes A =
\begin{pmatrix} 2 & 0 \\ 0 & -1 \end{pmatrix} \otimes \begin{pmatrix} 2 & 0 \\ 0 & -1 \end{pmatrix}
=
\begin{pmatrix}
4 & 0 & 0 & 0\\
0 & -2 & 0 & 0\\
0 & 0 & -2 & 0\\
0 & 0 & 0 & 1
\end{pmatrix}
$$

In [ ]:
always_id.add_snapshot()
dm = always_id.get_desity_matrix_from_stabilizers()
display(array_to_latex(dm, prefix=f"DM {i}= "))